## Step 4 - Implement the data transformation steps

The data transformation application will take one Sentinel-1 products (stageg-in during **Step 1** of this tutorial) and generate the backscatter using the Sentinel Application Platform (SNAP) Python bindings.


The data transformation steps above will use a Jupyter Notebook to:

* Read the Sentinel-1 product
* Application of orbit file 
* Border noise removal 
* Calibration
* Speckle filtering 
* Terrain correction
* Conversion to dB

At runtime this Jupyter Notebook will be instantiated and parametrized to process the Sentinel-1 archive. 

### Concepts in a Jupyter Notebook streaming executable

#### Service definition

The data transformation application will be exposed as a Web Processing Service. 

The definition of the Web Processing Service information such as the _title_ and the _abstract_ is done with a Python dictionary: 


In [4]:
service = dict([('title', 'WFP-01-01-01 Sentinel-1 backscatter timeseries'),
                ('abstract', 'WFP-01-01-01 Data transformation application - Sentinel-1 backscatter timeseries'),
                ('id', 'wfp-01-01-01')])

<div class="alert alert-info">

**Note:** As you will see in Step 8, the process identifier is set by the Production Centre. The 'id' set above is used to provide and identifier to the workflow

</div>

#### Parameter Definition

Data transformation application may have to expose parameters that can be changed via the Web Processing Service interface at submission time.

These parameters are defined using a Python dictionary that defines the parameter identifier, its title and abstract and finally its default value:

In [6]:
filterSizeX = dict([('id', 'filterSizeX'),
               ('value', '5'),
               ('title', 'Speckle-Filter filterSizeX'),
               ('abstract', 'Set the Speckle-Filter filterSizeX (defaults to 5)')])

To use the parameter value, symply do:


In [8]:
int(filterSizeX['value'])

5

#### Runtime parameter definition

Runtime parameters are mandatory and define those parameters whose values will be changed at runtime.

These are:
 
* `input_identifier` - this is the Sentinel-1 product identifier. At runtime its value is replaced with the Sentinel-1 product identifier being processed
* `input_reference` - this is the Sentinel-1 product catalogue entry URL. At runtime its value is also replaced with the Sentinel-1 product catalogue entry URL being processed
* `data_path` - this is the local path where the Sentinel-1 was staged-in in **step 1**. At runtime its value is replaced by a folder with an unique value

#### Publishing outputs or results

To publish results simply save the output or result locally.

#### Implementing the data transformation processing steps

* Define the **Service definition** with:

In [10]:
service = dict([('title', 'WFP-01-01-01 Sentinel-1 backscatter timeseries'),
                ('abstract', 'WFP-01-01-01 Data transformation application - Sentinel-1 backscatter timeseries'),
                ('id', 'wfp-01-01-01')])

Define the parameters:

In [11]:
filterSizeX = dict([('id', 'filterSizeX'),
               ('value', '5'),
               ('title', 'Speckle-Filter filterSizeX'),
               ('abstract', 'Set the Speckle-Filter filterSizeX (defaults to 5)')])

In [12]:
filterSizeY = dict([('id', 'filterSizeY'),
               ('value', '5'),
               ('title', 'Speckle-Filter filterSizeY'),
               ('abstract', 'Set the Speckle-Filter filterSizeY (defaults to 5)')])

In [13]:
polarisation = dict([('id', 'polarisation'),
               ('value', 'VV'),
               ('title', 'Sentinel-1 polarisation (VV or HH)'),
               ('abstract', 'Sentinel-1 polarisation (VV or HH)')])

In [14]:
wkt = dict([('id', 'wkt'),
            ('value', 'POLYGON((-5.5 17.26, -1.08 17.26, -1.08 13.5, -5.5 13.5, -5.5 17.26))'),
            ('title', 'Area of interest in WKT'),
            ('abstract', 'Area of interest using a polygon in Well-Known-Text format')])

* Define the **Runtime parameters**

In [15]:
input_identifier = 'S1A_IW_GRDH_1SDV_20171210T182024_20171210T182049_019644_021603_0A33'

In [16]:
input_reference = 'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_GRDH_1SDV_20171210T182024_20171210T182049_019644_021603_0A33' 

In [17]:
data_path = '/workspace/data'

* Define the Python libraries 

In [18]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")
import os
import sys
import glob
sys.path.append('/opt/anaconda/bin/')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import gc

* Read the Sentinel-1 product

In [20]:
s1meta = "manifest.safe"

s1prd = os.path.join(data_path, input_identifier, input_identifier + '.SAFE', s1meta)

reader = ProductIO.getProductReader("SENTINEL-1")
product = reader.readProductNodes(s1prd, None)

RuntimeError: java.io.IOException: org.esa.s1tbx.io.sentinel1.Sentinel1ProductReader
[input=/workspace/data/S1A_IW_GRDH_1SDV_20171210T182024_20171210T182049_019644_021603_0A33/S1A_IW_GRDH_1SDV_20171210T182024_20171210T182049_019644_021603_0A33.SAFE/manifest.safe]:
/workspace/data/S1A_IW_GRDH_1SDV_20171210T182024_20171210T182049_019644_021603_0A33/S1A_IW_GRDH_1SDV_20171210T182024_20171210T182049_019644_021603_0A33.SAFE/manifest.safe not found